# Subscriber

Subscriber to drugi obiekt, który obsługuje odczytywanie wiadomości pojawiających się na *topicu*.

Wiadomości przychodzą asynchronicznie tj. nie wiemy kiedy przyjdą, dlatego potrzebujemy funkcji w formie *callbacku* która obsłuży przychodzącą wiadomość (i coś z nią zrobi).

Zróbmy najprostszy callback który po prostu printuje wiadomosc *obrobka_wiadomosci*

In [2]:
from run_in_term import run_lxterminal

In [8]:
import rclpy
from rclpy.node import Node

from std_msgs.msg import String


class MinimalNode(Node):
    def __init__(self):
        super().__init__("minimal_subscriber")


rclpy.init()

minimal_node = MinimalNode()

In [9]:
def obrobka_wiadomosci(wiadomosc):
    print("przyszla wiadomosc", wiadomosc.data)

In [6]:
# Podobnie jak poprzednio, przy tworzeniu obiektu subscriber potrzebujemy powiedzieć jaki typ wiadomości i jaki topic
# subskrybujemy. Dodatkowym argumentem jest callback function tj. funkcja którą uruchomimy jak przyjdzie wiadomość

In [4]:
subscription = minimal_node.create_subscription(
    String, "wiadomosc", obrobka_wiadomosci, 10
)

# Spinujmy :-)
tak jak timer, musimy spinować aby przyjmować wiadomosci. Tym razem wykorzystamy spin_once, które po otrzymaniu jednej wiadomości 

Wykorzystując poprzedni notebook, wyślij jakąś wiadomość na ten topic.

Samo spinowanie jest blokujące i może nam utrudnić interakcje

In [10]:
rclpy.spin_once(minimal_node)

przyszla wiadomosc hello world5


# Spinujmy w odrębnym wątku 

Możemy skonstruować osobny wątek do obsługi spin

In [12]:
import threading


class ThreadedSpinner:
    def __init__(self, node: Node) -> None:
        self.node = node

    def __thread_target(self) -> None:
        while self.__thread_state:
            rclpy.spin_once(self.node, timeout_sec=0.1)

    def stop(self) -> None:
        self.__thread_state = False
        print("stopped spinning in thread")

    def spin_in_thread(self) -> None:
        self.__thread_state = True
        local_thread = threading.Thread(target=self.__thread_target)
        local_thread.start()

In [15]:
threaded_spinner = ThreadedSpinner(minimal_node)

In [16]:
threaded_spinner.spin_in_thread()

przyszla wiadomosc hello world5


Wyślij w osobnej konsoli

In [19]:
threaded_spinner.stop()
minimal_node.destroy_subscription(subscription)

In [ ]:
# Ćwiczenie: zapisz stan wiadomości do zmiennej (globalnej lub utwórz obiekt) najnowsza_wiadomosc.

print(najnowsza_wiadomosc)

In [18]:
# Ćwiczenie 2 podejrzyjmy wiadomość na kanale scan: jest to wiadomość typu sensor_msgs/msg/LaserScan
# skonstruuj subscribera i wyswietl

from sensor_msgs.msg import LaserScan


def wyswietl_laser(wiadomosc_laser):
    pass


subscriber_laser = ...

In [6]:
threaded_spinner = ThreadedSpinner(minimal_node)

In [1]:
#!pip install bqplot

In [2]:
%matplotlib inline

# Możemy nawet wizualizować dane bezpośrednio w Jupyterze
we can visualise the laser data even inside jupyter

In [46]:
from ipywidgets import FloatSlider

dystans_widget = FloatSlider(value=3.8, min=0, max=3.8)

dystans_widget

FloatSlider(value=3.8, max=3.8)

In [10]:
import bqplot.pyplot as plt
import numpy as np

fig = plt.figure(title="Laser Plot")

x = np.linspace(0, 360, 360)
y = np.sin(x)
curve = plt.plot(x, 10 * np.cos(y))

plt.show()

In [51]:
from sensor_msgs.msg import LaserScan

print(LaserScan().ranges)


def update_laser_plot(laser_data):
    global ranges
    global laser_data_g
    laser_data_g = laser_data

    ranges = np.array(laser_data.ranges)

    # filter infts
    ranges[~np.isfinite(ranges)] = laser_data.range_max  # set more than range

    with curve.hold_sync():
        curve.x = x
        curve.y = np.array(ranges)

    single_distance = ranges[0]

    dystans_widget.value = single_distance


subscriber_laser = minimal_node.create_subscription(
    LaserScan, "scan", update_laser_plot, 1
)

array('f')


In [34]:
laser_data_g.range_max

3.5

In [13]:
threaded_spinner = ThreadedSpinner(minimal_node)

In [52]:
threaded_spinner.spin_in_thread()

In [53]:
threaded_spinner.stop()
minimal_node.destroy_subscription(subscriber_laser)

stopped spinning in thread


True

In [4]:
# możemy też zobaczyć co jest najblizej nas

import tornado

print(tornado.__doc__)

The Tornado web server and tools.


# Możemy łączyć subskrybery i publishery
We can connect subscriber and publisher

Excercise:
Make the robot stop (send command velocity zero to cmd_vel when front distance to the obstacle is less than some value
Zatrzymaj robocika (poprzez cmd_vel) jeśli odległość od przeszkody jest mniejsza niż jakaś wartość

In [ ]:
# Excercise

from geometry_msgs.msg import Twist


zatrzymaj_predkosc = Twist()


publisher_cmd_vel = minimal_node.create_publisher(Twist, "cmd_vel", 10)

In [38]:
def laser_stopper(laser_data):
    ranges = np.array(laser_data.ranges)

    # filter infts
    ranges[~np.isfinite(ranges)] = laser_data.range_max  # set more than range

    front_distance = ranges[0]

    # make reasonable rule
    # if
    # publisher_cmd_vel.publish(zatrzymaj_predkosc)

In [ ]:
subscriber_laser_stopper = minimal_node.create_subscription(
    LaserScan, "scan", laser_stopper, 1
)

In [ ]:
threaded_spinner.spin_in_thread()

In [ ]:
threaded_spinner.stop()
minimal_node.destroy_subscription(subscriber_laser_stopper)

Teraz np. z poprzedniego notebooka wyślij robota na spotkanie ze ścianą

[Next exercise: 5. ROS Service](5.%20ROS%20Service.ipynb)